In [8]:
# import libraries
# psycopg2 allows to work with PostgreSQL
import psycopg2
from datetime import datetime

# DB configuration 
DB_NAME = "beam"
DB_USER = "chris"
DB_PASSWORD = "" 
DB_HOST = "localhost"
DB_PORT = "5432"
TABLE_NAME = '"Observations"."ObservationInfo"'

# txt file with observation
TXT_FILE = "/Users/chris/Desktop/BEAM/Database/observation_metadata_2025-08-07_19-21-33.txt"

# parse text file to the data dictionary
def parse_metadata(filepath):
    data = {}
    with open(filepath, "r") as file:
        for line in file:
            if ":" in line:
                key, value = line.strip().split(":", 1)
                data[key.strip()] = value.strip().strip('"')

    # handle datetime parsing
    obs_start_str = data.get("Observation_Start")
    obs_end_str = data.get("Observation_End")

    obs_start_dt = datetime.strptime(obs_start_str, "%Y-%m-%d_%H-%M-%S")
    obs_end_dt = datetime.strptime(obs_end_str, "%Y-%m-%d_%H-%M-%S")

    return {
        "obs_date": obs_start_dt.date(),
        "obs_start_time": obs_start_dt.time(),
        "obs_end_time": obs_end_dt.time(),
        "receiver": data.get("Receiver"),
        "rest_freq_MHz": float(data.get("rest_freq_MHz")),
        "bandwidth_MHz": float(data.get("Bandwidth_MHz")),
        "ra_deg": float(data.get("ra_deg")),
        "dec_deg": float(data.get("dec_deg")),
        "integration_time_s": float(data.get("integration_time_s")),
        "observation_type": data.get("observation_type"),
        "Object": data.get("Object"),
        "rf_gain": float(data.get("rf_gain")),
        "if_gain": float(data.get("if_gain")),
        "bb_gain": float(data.get("bb_gain")),
    }

# insert the new row(observation) to the DB
def insert_into_db(metadata):
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        cursor = conn.cursor()

        sql = f"""
            INSERT INTO {TABLE_NAME} (
                obs_date, obs_start_time, obs_end_time,
                receiver, "rest_freq_MHz", "bandwidth_MHz",
                ra_deg, dec_deg, integration_time_s,
                observation_type, "Object", rf_gain, if_gain, bb_gain
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        values = (
            metadata["obs_date"],
            metadata["obs_start_time"],
            metadata["obs_end_time"],
            metadata["receiver"],
            metadata["rest_freq_MHz"],
            metadata["bandwidth_MHz"],
            metadata["ra_deg"],
            metadata["dec_deg"],
            metadata["integration_time_s"],
            metadata["observation_type"],
            metadata["Object"],
            metadata["rf_gain"],
            metadata["if_gain"],
            metadata["bb_gain"]
        )

        cursor.execute(sql, values)
        conn.commit()
        cursor.close()
        conn.close()
        print("Observation inserted successfully.")
    except Exception as e:
        print("Error inserting data:", e)

# RUN
if __name__ == "__main__":
    metadata = parse_metadata(TXT_FILE)
    insert_into_db(metadata)



Observation inserted successfully.
